In [218]:
import numpy as np 
import pandas as pd 

# Data preprocessing

In [219]:
train_data = pd.read_csv("data/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [220]:
test_data = pd.read_csv("data/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [221]:
gender_submission = pd.DataFrame(test_data['PassengerId'])

In [222]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [223]:
from sklearn.ensemble import RandomForestClassifier as RFC

In [224]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [225]:
X_train_df = train_data.drop(columns = ['Survived'])
y_train = train_data['Survived']

In [226]:
X_test_df = test_data
# y_test = test_data['Survived'] # This part does not exist ... yet!

In [227]:
from sklearn.preprocessing import LabelEncoder
X_train_df[X_train_df.columns]=X_train_df[X_train_df.columns].apply(LabelEncoder().fit_transform)
X_train_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,2,108,1,28,1,0,523,18,147,2
1,1,0,190,0,51,1,0,596,207,81,0
2,2,2,353,0,34,0,0,669,41,147,2
3,3,0,272,0,47,1,0,49,189,55,2
4,4,2,15,1,47,0,0,472,43,147,2
...,...,...,...,...,...,...,...,...,...,...,...
886,886,1,548,1,35,0,0,101,85,147,2
887,887,0,303,0,24,0,0,14,153,30,2
888,888,2,413,0,88,1,2,675,131,147,2
889,889,0,81,1,34,0,0,8,153,60,0


In [228]:
X_test_df[X_test_df.columns]=X_test_df[X_test_df.columns].apply(LabelEncoder().fit_transform)
X_test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,2,206,1,44,0,0,152,24,76,1
1,1,2,403,0,60,1,0,221,5,76,2
2,2,1,269,1,74,0,0,73,41,76,1
3,3,2,408,1,34,0,0,147,34,76,2
4,4,2,178,0,27,1,1,138,46,76,2
...,...,...,...,...,...,...,...,...,...,...,...
413,413,2,353,1,79,0,0,267,31,76,2
414,414,0,283,0,51,0,0,324,154,22,0
415,415,2,332,1,50,0,0,346,9,76,2
416,416,2,384,1,79,0,0,220,31,76,2


# Scaling the data

In [229]:
from sklearn.preprocessing import StandardScaler as SS

In [230]:
ss = SS()
X_train_s = ss.fit_transform(X_train_df)
X_test_s = ss.transform(X_test_df)

In [231]:
X_train_s

array([[-1.73010796,  0.82737724, -1.31021659, ..., -1.24717035,
         0.45150219,  0.58111394],
       [-1.72622007, -1.56610693, -0.99141018, ...,  1.45116508,
        -1.38162664, -1.93846038],
       [-1.72233219,  0.82737724, -0.35768524, ..., -0.91880149,
         0.45150219,  0.58111394],
       ...,
       [ 1.72233219,  0.82737724, -0.12441226, ...,  0.36612014,
         0.45150219,  0.58111394],
       [ 1.72622007, -1.56610693, -1.41518943, ...,  0.6802121 ,
        -1.9648949 , -1.93846038],
       [ 1.73010796,  0.82737724, -0.87477369, ..., -1.07584746,
         0.45150219, -0.67867322]])

In [232]:
rfc = RFC()

In [233]:
from sklearn.ensemble import RandomForestClassifier as RFC
rfc.fit(X_train_s, y_train)
rfc.score(X_train_s, y_train)

1.0

In [234]:
rfc.predict(X_train_s)

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

### The 1.0 score for train seems suspicious ... maybe my forest is overfit? Let's check out what is going on under the hood

In [235]:
def compare_classes(actual, predicted, names=None):
    '''Function returns a confusion matrix, and overall accuracy given:
            Input:  actual - a list of actual classifications
                    predicted - a list of predicted classifications
                    names (optional) - a list of class names
    '''
    accuracy = sum(actual==predicted)/actual.shape[0]
    
    classes = pd.DataFrame(columns = ['Actual', 'Predicted'])
    classes['Actual'] = actual
    classes['Predicted'] = predicted

    conf_mat = pd.crosstab(classes['Actual'], classes['Predicted'])
    
    if type(names) != type(None):
        conf_mat.index = names
        conf_mat.index.name = 'Actual'
        conf_mat.columns = names
        conf_mat.columns.name = 'Predicted'
    
    print('Accuracy = ' + format(accuracy, '.2f'))
    return conf_mat, accuracy

In [236]:
def do_Kfold(model,X,y,k,scaler = None, random_state = 146):
    
    
    
    from sklearn.model_selection import KFold
    
    kf = KFold(n_splits=k, random_state = random_state, shuffle=True) 
   

    train_scores = [] 
    test_scores = []

    for idxTrain, idxTest in kf.split(X): 
        
        Xtrain = X[idxTrain, :] 
        Xtest = X[idxTest, :] 
        ytrain = y[idxTrain] 
        ytest = y[idxTest] 
        if scaler != None: 
            Xtrain = scaler.fit_transform(Xtrain)
            Xtest = scaler.transform(Xtest) 

        model.fit(Xtrain,ytrain) 

        train_scores.append(model.score(Xtrain,ytrain)) 
        test_scores.append(model.score(Xtest,ytest)) 
        
    return train_scores, test_scores

In [237]:
y_pred = rfc.predict(X_test_s)
cm,acc = compare_classes(y_pred,y_pred)
cm

Accuracy = 1.00


Predicted,0,1
Actual,,
0,233,0
1,0,185


### The predictions seem normal for the testing data, lets look at the features

In [238]:
importances = rfc.feature_importances_
importances

array([0.10112527, 0.05801683, 0.11094138, 0.23992733, 0.10564108,
       0.02730924, 0.02021292, 0.13395433, 0.11539285, 0.06714946,
       0.0203293 ])

### The fourth feature (gender) has the highest importance, which makes sense given the fact that the women left the ship first.

### Now we are creating a mini train test k-fold cross validation within the training data given

In [239]:
k = 10
rfc = RFC()
train_scores, test_scores = do_Kfold(rfc, X_train_s, y_train, k, ss)

In [240]:
train_scores

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [241]:
test_scores

[0.7888888888888889,
 0.8539325842696629,
 0.8426966292134831,
 0.8426966292134831,
 0.7752808988764045,
 0.8426966292134831,
 0.7865168539325843,
 0.8764044943820225,
 0.8876404494382022,
 0.8539325842696629]

In [242]:
rfc.feature_importances_

array([0.10262121, 0.06200429, 0.11388325, 0.23111723, 0.10764395,
       0.02518669, 0.02497141, 0.13134167, 0.11641545, 0.06460664,
       0.02020819])

### Still the same importances basically

We see that the fourth feature (gender) is the most important as expected.

### Let's optimize some hyperparameters to hopefully help diminish overfitting

In [243]:
from sklearn.model_selection import KFold, GridSearchCV

param_grid = dict(n_estimators=[1,100,500],max_depth = [2,3,4,5],min_samples_split = [2,3,4,5]) #params to test
cv = KFold(n_splits=10, random_state=146, shuffle = True)
grid = GridSearchCV(RFC(random_state = 146), param_grid=param_grid, cv=cv, scoring='accuracy')

#### A single mini train test split within the training data given

In [244]:
from sklearn.model_selection import train_test_split as tts
mini_X_train, mini_X_test, mini_y_train, mini_y_test = tts(X_train_s, y_train, test_size=0.4, random_state = 146) 

In [245]:
grid.fit(mini_X_train, mini_y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=146, shuffle=True),
             estimator=RandomForestClassifier(random_state=146),
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [1, 100, 500]},
             scoring='accuracy')

In [246]:
print(f'The best parameters are {grid.best_params_} with a score of {grid.best_score_:.2f}')

The best parameters are {'max_depth': 4, 'min_samples_split': 5, 'n_estimators': 500} with a score of 0.81


In [247]:
rfc = RFC(n_estimators = grid.best_params_['max_depth'], max_depth = grid.best_params_['min_samples_split'], min_samples_split = grid.best_params_['n_estimators'])
rfc.fit(mini_X_train, mini_y_train)
print(rfc.score(mini_X_train, mini_y_train))
print(rfc.score(mini_X_test, mini_y_test))

0.6048689138576779
0.6330532212885154


### Let's go back to fitting all of the data

In [248]:
grid.fit(X_train_s, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=146, shuffle=True),
             estimator=RandomForestClassifier(random_state=146),
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [1, 100, 500]},
             scoring='accuracy')

In [249]:
print(f'The best parameters are {grid.best_params_} with a score of {grid.best_score_:.2f}')

The best parameters are {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 500} with a score of 0.83


In [250]:
rfc = RFC(n_estimators = grid.best_params_['max_depth'], max_depth = grid.best_params_['min_samples_split'], min_samples_split = grid.best_params_['n_estimators'])
rfc.fit(X_train_s, y_train)
rfc.score(X_train_s, y_train)

0.77665544332211

In [251]:
rfc.feature_importances_

array([0. , 0.2, 0. , 0.2, 0.2, 0. , 0. , 0. , 0.2, 0. , 0.2])

### Let's compare back to the possibly overfit model with no specified parameters

In [252]:
rfc = RFC()
rfc.fit(X_train_s, y_train)
rfc.score(X_train_s, y_train)

1.0

In [253]:
rfc.feature_importances_

array([0.10190377, 0.06167071, 0.11445649, 0.23999096, 0.10595928,
       0.03007211, 0.02320206, 0.12992106, 0.11666851, 0.05591119,
       0.02024387])

#### Although the model has signs of overfitting, the diverse state of the feature importance is an indicator that the possibly overfit model is the best bet. The 'optimized' model (through grid search) eliminates too many variables and often includes variables at 0.4, and 0.2, etc., which shows that the feature importance for the optimized model is not accurate to what real world data seems to be. Real world data is much more complicated than 0.4 and 0.2 and 0.6 importance. The basic model shows just that.

### Here are the model's final predictions on survival of passengers

In [254]:
y_final_predictions = rfc.predict(X_test_s)
y_final_predictions

array([0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

### Placing it in the gender_submission-like file as requested

In [255]:
gender_submission

,PassengerId
0,892
1,893
2,894
3,895
4,896
...,...
413,1305
414,1306
415,1307
416,1308


In [256]:
gender_submission['Survived'] = y_final_predictions
Titanic_submission = gender_submission
Titanic_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [258]:
Titanic_submission.to_csv('data/Titanic_submission.csv', sep=',', index=False, encoding='utf-8')

# Conclusions:



#### We can conclude with high likelihood from this analysis that the gender of the passenger was the most important feature in determining survival rate, followed by other variables that have relatively similar importances, because their ranks vary depending on the random state you put.

# Update:

Score: 0.71291

Your score represents your submission's accuracy. For example, a score of 0.7 in this competition indicates you predicted Titanic survival correctly for 70% of people.